### **API SADVR - Portrait statistique**  
https://www.cen.umontreal.ca/espacedoc/sadvr/  

Le Service d’accès aux données de la Vitrine-Recherche (SADVR) est un service institutionnel de partage de données relatives aux profils des professeur·e·s et à leurs activités académiques ainsi qu'aux expertises et disciplines de recherche des facultés, départements et centres de recherche affiliés à l'Université.

---

#### **Professeurs**

In [10]:
import requests
import json
import pandas as pd
from ast import literal_eval
from collections import Counter

Import des données

In [11]:
baseURI = 'https://www.recherche.umontreal.ca/vitrine/rest/api/1.7/umontreal'

# uri id/individu
uri = f'{baseURI}/id/individu'
res = json.loads(requests.get(uri).text)
dataIndividus = pd.DataFrame(res['data'])

# requête SOLR dans le répertoire des professeurs
index = 0
res = json.loads(requests.get(f'{baseURI}/recherche/professeur/select?q=ID:*&start={index}').text)
nbResults = res['paginationSOLR']['numFound']

dataProfs = []
for i in range(0, nbResults, 20):
    res = json.loads(requests.get(
        f'{baseURI}/recherche/professeur/select?q=ID:*&start={index}&rows=20'
        ).text)['data']
    
    dataProfs += res
    index += 20

dataProfs = pd.DataFrame(dataProfs)

# uri /info/individu


Normalisation / filtrage

In [12]:
dataIndividus = dataIndividus.explode('etablissementsAffilies').reset_index(drop=True)
dataIndividus['etablissementAffilie'] = dataIndividus['etablissementsAffilies'].apply(
    lambda x: x['nom'] if isinstance(x, dict) else None)

dataIndividus = dataIndividus.drop(columns=
        ['etablissementsAffilies', 'fonction', 'courriel', 'visuelOfficiel', 'visuelOfficielGrand']
    ).drop_duplicates()

dataIndividus = dataIndividus[dataIndividus['nom'] != '?_?']

# Séparer les colonnes qui contiennent des données structurées en JSON en muliples colonnes distinctes
def explodeNormalize(df: pd.DataFrame, columns: list, ):
    """
    Cette fonction prend en paramètre un DataFrame et une liste contenant les noms des colonnes à normaliser.
    Elle retourne le DataFrame modifié
    """
    for col in columns:
        dTypeCol = Counter(df[col].apply(lambda x: type(literal_eval(str(x)))).tolist()).most_common(1)[0][0]
        if dTypeCol == list:
            df = df.explode(col).reset_index(drop=True)

        dfTemp = pd.json_normalize(df[col]).add_prefix(f'{col}.') 
        
        df = pd.concat([df, dfTemp], axis=1).drop(col, axis=1)

    return df

# Colonnes à normaliser
columns = ['affiliations', 'expertise', 'expertise.secteursRecherche',
    'expertise.disciplines', 'expertise.motsCles', 'expertise.periodesChronologiques',
    'expertise.phraseCle', 'expertise.continents']

dataProfs = explodeNormalize(dataProfs, columns)

dataProfs = dataProfs.drop(
    columns=["prenom", "nom", "etablissementsAffilies", "visuelOfficiel", "visuelOfficielGrand",
      "courriels", "telephones", "urlVitrine", "nomSAD", "prenomSAD", "in-memoriam",
      "affiliations.courrielInstitutionnel", "affiliations.local", "affiliations.exclusion", "affiliations.exclusionTel", 
      "affiliations.fonction.nom", "affiliations.telephone.numero", "affiliations.telephone.poste"])

data = dataIndividus.merge(dataProfs, how='right')
data

,idsadvr,nom,prenom,fonctionCodeSad,uniteAdmin,in-memoriam,etablissementAffilie,recrutementRecherche,affiliations.immeuble,affiliations.fonction.codeSad,...,expertise.periodesChronologiques.uid,expertise.periodesChronologiques.ordre,expertise.periodesChronologiques.codeLangue,expertise.periodesChronologiques.nom,expertise.phraseCle.codeLangue,expertise.phraseCle.contenu,expertise.continents.uid,expertise.continents.ordre,expertise.continents.codeLangue,expertise.continents.nom
0,in13580,Sygusch,Jurgen,42001,Faculté de médecine - Département de biochimie...,False,None,False,Roger-Gaudry,42001,...,NaN,NaN,NaN,NaN,fre,Analyse structurale des macromolécules et sign...,NaN,NaN,NaN,NaN
1,in13580,Sygusch,Jurgen,42001,Faculté de médecine - Département de biochimie...,False,None,False,Roger-Gaudry,42001,...,NaN,NaN,NaN,NaN,fre,Analyse structurale des macromolécules et sign...,NaN,NaN,NaN,NaN
2,in13580,Sygusch,Jurgen,42001,Faculté de médecine - Département de biochimie...,False,None,False,Roger-Gaudry,42001,...,NaN,NaN,NaN,NaN,fre,Analyse structurale des macromolécules et sign...,NaN,NaN,NaN,NaN
3,in13580,Sygusch,Jurgen,42001,Faculté de médecine - Département de biochimie...,False,None,False,Roger-Gaudry,42001,...,NaN,NaN,NaN,NaN,fre,Analyse structurale des macromolécules et sign...,NaN,NaN,NaN,NaN
4,in13580,Sygusch,Jurgen,42001,Faculté de médecine - Département de biochimie...,False,None,False,Roger-Gaudry,42001,...,NaN,NaN,NaN,NaN,fre,Analyse structurale des macromolécules et sign...,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2958423,in22406,Martinez,Marina,52061,Faculté de médecine - Département de neuroscie...,False,None,False,Pavillon Paul-G.-Desmarais,52061,...,NaN,NaN,NaN,NaN,fre,Réadaptation de la marche après des lésions mé...,NaN,NaN,NaN,NaN
2958424,in22406,Martinez,Marina,52061,Faculté de médecine - Département de neuroscie...,False,None,False,Pavillon Paul-G.-Desmarais,52061,...,NaN,NaN,NaN,NaN,fre,Réadaptation de la marche après des lésions mé...,NaN,NaN,NaN,NaN
2958425,in22406,Martinez,Marina,52061,Faculté de médecine - Département de neuroscie...,False,None,False,Pavillon Paul-G.-Desmarais,52061,...,NaN,NaN,NaN,NaN,fre,Réadaptation de la marche après des lésions mé...,NaN,NaN,NaN,NaN
2958426,in22406,Martinez,Marina,52061,Faculté de médecine - Département de neuroscie...,False,None,False,Pavillon Paul-G.-Desmarais,52061,...,NaN,NaN,NaN,NaN,fre,Réadaptation de la marche après des lésions mé...,NaN,NaN,NaN,NaN


In [14]:
data.to_csv('./tables/SADVR_professeurs.csv', index=False)